## Calling C functions from Python

In [ ]:
import ctypes

Here's a library everybody has:

In [ ]:
libc = ctypes.cdll.LoadLibrary("libc.so.6")

And here's a function call in it. The `None` is passing NULL as the first argument, since we don't want to reuse a `time_t` integer.

```c
time_t     time(time_t *);
```

In [ ]:
libc.time(None)

It's where Python gets the decimal part of its `time.time()` from.

In [ ]:
import time
libc.time(None), time.time()

Here's a challenge: where does this text go?

```c
int     printf(const char * format, ...);
```

In [ ]:
libc.printf("Hello from C!\n")

Some Python types are automatically converted into the correct C type, but if unsure, wrap with a `ctypes` type.

| ctypes type | C type | Python type |
|:------------|:-------|:------------|
| `c_bool` | `_Bool` | `bool` |
| `c_char` | `char` | `1-character string` |
| `c_wchar` | `wchar_t` | `1-character unicode string` |
| `c_byte` | `char` | `int/long` |
| `c_ubyte` | `unsigned char` | `int/long` |
| `c_short` | `short` | `int/long` |
| `c_ushort` | `unsigned short` | `int/long` |
| `c_int` | `int` | `int/long` |
| `c_uint` | `unsigned int` | `int/long` |
| `c_long` | `long` | `int/long` |
| `c_ulong` | `unsigned long` | `int/long` |
| `c_longlong` | `__int64` | `int/long` |
| `c_ulonglong` | `unsigned __int64` | `int/long` |
| `c_float` | `float` | `float` |
| `c_double` | `double` | `float` |
| `c_longdouble` | `long double` | `float` |
| `c_char_p` | `char*` (NULL terminated) | `string` or `None` |
| `c_wchar_p` | `wchar_t*` (NULL terminated) | `unicode` or `None` |
| `c_void_p` | `void*` | `int/long` or `None` |

In [ ]:
libc.printf("one %d two %g three %s\n",
            ctypes.c_int(1),
            ctypes.c_double(2.2),
            ctypes.c_char_p("THREE"))

**Okay: not fair.** The answer is that the text gets printed out in the terminal where Jupyter is running. If you were running this on your laptop (my original intention), you'd see it in the terminal that's running Jupyter. But now that it's running in AWS and you're not even ssh'ed into AWS, you'll never see it.

## Calling Python from C

The [GNU Scientific Library (GSL)](https://www.gnu.org/software/gsl/) is a large mathematical library.

In [ ]:
gslcblas = ctypes.CDLL("libgslcblas.so", mode=ctypes.RTLD_GLOBAL)
gsl = ctypes.CDLL("libgsl.so")

Many of its functions take functions as arguments (numerical differentiation, integration, root finding, ...), so it has a common interface for function arguments.

```c
struct gsl_function {
    double (* function) (double x, void * params);
    void * params;  };
```

We can make this structure in ctypes because it has methods for
   * function types (`ctypes.CFUNCTYPE(rettype, argtypes...)`)
   * pointers (`ctypes.POINTER(type)`)
   * structs (`ctypes.Structure` with a `_fields_` member)

Just be aware that these ctypes functions make _type_ objects, which then have to be called to make _objects._ You'll often do `something(typeargs)(objectargs)`.

See the [documentation](https://docs.python.org/2/library/ctypes.html) for the full story.

In [ ]:
# my function
def f(x, params):
    out = 3.0*x**2 + 2.0*x + 1.0
    print "f({0}) = {1}".format(x, out)
    return out

# callback with the appropriate signature
func_type = ctypes.CFUNCTYPE(ctypes.c_double, ctypes.c_double, ctypes.POINTER(None))

# wrapped up in the struct that GSL expects
class gsl_function(ctypes.Structure):
    _fields_ = [("f", func_type),
                ("params", ctypes.POINTER(None))]

# finally, make the object
callback = gsl_function(func_type(f), 0)

In [ ]:
result = ctypes.c_double(-1.0);
abserr = ctypes.c_double(-1.0);

p_result = ctypes.POINTER(ctypes.c_double)(result)
p_abserr = ctypes.POINTER(ctypes.c_double)(abserr)

In [ ]:
gsl.gsl_deriv_central(ctypes.POINTER(gsl_function)(callback), ctypes.c_double(2.0), ctypes.c_double(1e-8), p_result, p_abserr)

In [ ]:
p_result[0], p_abserr[0]

## The snake eats its own tail

On Monday, I added functional methods to the `list` and `generator` types. Usually, you can't add methods to Python builtin types. To make it work, we can use ctypes to alter Python's own C code.

In [ ]:
if hasattr(ctypes.pythonapi, "Py_InitModule4_64"):
    Py_ssize_t = ctypes.c_int64   # modern versions of Python 2 use 64-bit ints almost everywhere
else:
    Py_ssize_t = ctypes.c_int     # very old versions of Python used plain C ints

class PyObject(ctypes.Structure): pass
PyObject._fields_ = [("ob_refcnt", Py_ssize_t),
                     ("ob_type", ctypes.POINTER(PyObject))]

This "PyObject" represents the C struct called "PyObject" in the C API. All Python objects have the following C struct head:

```c
struct PyObject {
    Py_size_t ob_refcnt;     // number of references (for GC)
    PyObject *ob_type;       // the object's type (always a
};                           // runtime thing in Python)
```

In [ ]:
class SlotsPointer(PyObject):
    _fields_ = [("dict", ctypes.POINTER(PyObject))]

def proxy_builtin(cls):
    name = cls.__name__
    slots = getattr(cls, "__dict__", name)

    pointer = SlotsPointer.from_address(id(slots))
    namespace = {}

    ctypes.pythonapi.PyDict_SetItem(
        ctypes.py_object(namespace),
        ctypes.py_object(name),
        pointer.dict
    )

    return namespace[name]

In [ ]:
# don't call this twice (you'll crash the interpreter)

def do_crazy_things(lst):
    print "crazy {0}".format(lst)

proxy_builtin(list)["crazy"] = property(do_crazy_things)
hasattr([], "crazy")

In [ ]:
[1, 2, 3, 4, 5].crazy